# 2. Schools Data: INEP & Censo Escolar

**The Problem:** School data is split. One file has the address (lat/lon); another massive file has the metrics (students, staff, labs).

**The Solution:** AtlasBR performs an automatic **Data Fusion**, giving you a single GeoDataFrame ready for mapping.

**What you will learn:**
1.  **Rich Points:** Loading schools with attached metrics.
2.  **Filtering:** Analyzing the Public vs. Private divide.


In [1]:
import sys
import os
from pathlib import Path

# --- DEVELOPER SETUP (Optional) ---
# If running locally without 'pip install', we add the '../src' folder to path.
current_path = Path(os.getcwd())
if current_path.name == "tutorials":
    # Go up one level to root, then into 'src' (if using src-layout) or just root (flat-layout)
    root_dir = current_path.parent
    src_dir = root_dir / "src"
    
    if src_dir.exists():
        sys.path.append(str(src_dir))
    else:
        sys.path.append(str(root_dir))

import atlasbr

In [2]:
from atlasbr.app.inep import load_schools

atlasbr.configure_logging()
MUNICIPALITY = "Florianópolis, SC"

## 1. The Fusion Load
We request schools for 2023. Under the hood, AtlasBR is joining `br_bd_diretorios_brasil.escola` with `br_inep_censo_escolar`.

In [3]:
gdf_schools = load_schools(
    places=[MUNICIPALITY],
    year=2023,
    as_gdf=True,  # Return as Points (False returns raw Table)
    gcp_billing=os.getenv("GCLOUD_PROJECT_ID") # Required for BD
)

print(f"🎓 Found {len(gdf_schools)} schools.")
display(gdf_schools.head(3))


2025-12-16 19:35:01,859 - atlasbr - INFO -     🌍 Fetching municipality metadata from geobr...
2025-12-16 19:35:02,557 - atlasbr - INFO -     ℹ️  Resolved 'Florianópolis, SC' -> 4205407
2025-12-16 19:35:03,633 - atlasbr - INFO -     🎓 Fetching Schools 2023 from Base dos Dados...


Downloading: 100%|██████████|

2025-12-16 19:35:05,705 - atlasbr - INFO -     🌍 Converting 244 schools to geometry...
2025-12-16 19:35:05,720 - atlasbr - INFO - ✅ Loaded 244 schools.



🎓 Found 244 schools.


,id_escola,id_municipio,dependencia_administrativa,etapas_modalidades_oferecidas,endereco,latitude,longitude,rede,quantidade_matricula_infantil,quantidade_matricula_fundamental,quantidade_matricula_medio,quantidade_docente_educacao_basica,quantidade_profissional,geometry
0,42000351,4205407,Federal,"Ensino Médio, Educação Profissional","AVENIDA MAURO RAMOS, 950 CENTRO. 88020-300 Flo...",-27.5939087430272,-48.5428133793627,Publica,0,0,1162,259,160,POINT (-48.54281 -27.59391)
1,42113083,4205407,Privada,Ensino Médio,"RUA ARCIPRESTE PAIVA, 110 CENTRO. 88010-530 Fl...",-27.5959489,-48.5491457,Privada,0,0,647,43,0,POINT (-48.54915 -27.59595)
2,42125901,4205407,Privada,Ensino Médio,"RUA TRAJANO, 100 CENTRO. 88010-010 Florianópol...",-27.59754,-48.55088,Privada,0,0,238,20,7,POINT (-48.55088 -27.59754)


## 2. Public vs. Private Network
AtlasBR standardizes the `rede` column (State, Municipal, Federal, Private) to simplify analysis.

In [4]:
# Let's count the split
print(gdf_schools["rede"].value_counts())

rede
Publica    153
Privada     91
Name: count, dtype: int64


## 3. Interactive Map: The Education Landscape
Zoom in to see where schools are clustered. Click on a point to see the student count.
* **Blue:** Public Schools
* **Red:** Private Schools

In [5]:

# Simple color mapping helper
def color_school(val):
    return "blue" if val == "Publica" else "red"

gdf_schools["color"] = gdf_schools["rede"].apply(color_school)

gdf_schools.explore(
    column="rede",
    cmap="Set1", # Categorical Colormap
    marker_kwds={"radius": 6},
    tooltip=[
        "id_escola", 
        "rede", 
        "quantidade_matricula_fundamental", 
        "quantidade_docente_educacao_basica"
    ],
    tiles="CartoDB Positron"
)